#Install Dependencies

In [1]:
%pip install fastapi uvicorn pyngrok nest-asyncio

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 11.2 MB/s  0:00:00

   -- -------------------------------------  1/16 [sniffio]
   ----- ----------------------------------  2/16 [PyYAML]
   ----- ----------------------------------  2/16 [PyYAML]
   ------- --------------------------------  3/16 [idna]
   ------- --------------------------------  3/16 [idna]
   ---------- -----------------------------  4/16 [h11]
   ---------- -----------------------------  4/16 [h11]
   ------------ ---------------------------  5/16 [click]
   ------------ ---------------------------  5/16 [click]
   ------------ ---------------------------  5/16 [click]
   ------------ ---------------------------  5/16 [click]
   -------------------- -------------------  8/16 [uvicorn]
   -------------------- -------------------  8/16 [uvicorn]
   -------------------- -------------------  8/16 [uvicorn]
   -------------------- ----------------

#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#Tenby10 Backend Experiment
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [2]:
%pip install cerebras-cloud-sdk pandas gradio

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------- ----------------------------- 2.9/11.0 MB 15.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.0 MB 16.1 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 17.2 MB/s  0:00:00
   ---------------------------------------- 0.0/63.5 MB ? eta -:--:--
   --- ------------------------------------ 5.5/63.5 MB 25.8 MB/s eta 0:00:03
   ------ --------------------------------- 10.5/63.5 MB 25.2 MB/s eta 0:00:03
   --------- ------------------------------ 15.2/63.5 MB 23.9 MB/s eta 0:00:03
   ------------- -------------------------- 21.0/63.5 MB 24.6 MB/s eta 0:00:02
   ----------------- ---------------------- 27.3/63.5 MB 25.4 MB/s eta 0:00:02
   ------------------- -------------------- 31.2/63.5 MB 24.7 MB/s eta 0:00:02
   ---------------------- ----------------- 36.2/63.5 MB 24.5 MB/s eta 0:00:

In [3]:
import os
from cerebras.cloud.sdk import Cerebras
import pandas as pd
import json
import gradio as gr


c:\Users\Kevin\IdeaProjects\TenBy10AnkiGenAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
client = Cerebras(api_key=os.environ["CEREBRAS_API_KEY_JOSEPH"])

def generate_flashcards(vocab_list, target_language="Spanish"):
    results = []
    has_reading = target_language.lower() in ["japanese", "chinese"]

    for word in vocab_list:
        # Prompt asks for reading if Japanese or Chinese
        if has_reading:
            prompt = f"""
            You are a multilingual language learning assistant.
            For the term \"{word}\", translate it into {target_language}, give the pronunciation (reading, 
            such as pinyin for Chinese or furigana for Japanese), and provide one natural example sentence in the input and target language.
            Respond ONLY in valid JSON with this exact structure:
            {{
                "term": "{word}",
                "translation": "...",
                "reading": "...",
                "example_sentence": "..."
            }}
            """
        else:
            prompt = f"""
            You are a multilingual language learning assistant.
            For the term \"{word}\", translate it into {target_language}, and give one natural example sentence in the input language and target language.
            Respond ONLY in valid JSON with this exact structure:
            {{
                "term": "{word}",
                "translation": "...",
                "example_sentence": "..."
            }}
            """
        try:
            completion = client.chat.completions.create(
                model="qwen-3-235b-a22b-instruct-2507",
                messages=[
                    {"role": "system", "content": "You are a helpful multilingual assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_completion_tokens=500,
                temperature=0.7,
                top_p=0.9,
                stream=False
            )
            content = completion.choices[0].message.content.strip()
            try:
                data = json.loads(content)
            except:
                # Fallback: handle JSON errors
                if has_reading:
                    data = {
                        "term": word,
                        "translation": content.split("\n")[0] if "\n" in content else content,
                        "reading": "",
                        "example_sentence": ""
                    }
                else:
                    data = {
                        "term": word,
                        "translation": content.split("\n")[0] if "\n" in content else content,
                        "example_sentence": ""
                    }
            if has_reading:
                results.append({
                    "Grammar/Vocab": data.get("term", word),
                    "Reading": data.get("reading", ""),
                    "Meaning": data.get("translation", ""),
                    "Example Sentence": data.get("example_sentence", "")
                })
            else:
                results.append({
                    "Grammar/Vocab": data.get("term", word),
                    "Meaning": data.get("translation", ""),
                    "Example Sentence": data.get("example_sentence", "")
                })
        except Exception as e:
            if has_reading:
                results.append({
                    "Grammar/Vocab": word,
                    "Reading": "",
                    "Meaning": f"Error: {e}",
                    "Example Sentence": ""
                })
            else:
                results.append({
                    "Grammar/Vocab": word,
                    "Meaning": f"Error: {e}",
                    "Example Sentence": ""
                })
    return pd.DataFrame(results)

In [5]:
def run_flashcard_generator(text, language):
    vocab_list = [w.strip() for w in text.split("\n") if w.strip()]
    df = generate_flashcards(vocab_list, language)
    csv_path = "anki_flashcards.csv"

    # Determine columns by language
    if language.lower() in ["japanese", "chinese"]:
        display_cols = ["Grammar/Vocab", "Reading", "Meaning", "Example Sentence"]
    else:
        display_cols = ["Grammar/Vocab", "Meaning", "Example Sentence"]
        # If DataFrame has Reading, drop it for display and CSV
        if "Reading" in df.columns:
            df = df.drop(columns=["Reading"])
    df = df.reindex(columns=display_cols)
    df.to_csv(csv_path, index=False)
    return df, csv_path

demo = gr.Interface(
    fn=run_flashcard_generator,
    inputs=[
        gr.Textbox(label="Enter vocab terms (one per line)", lines=6, placeholder="e.g.\nla organización\nel animal\nla tecnología"),
        gr.Radio(["Spanish", "French", "Japanese", "Chinese", "English"], label="Target Language")
    ],
    outputs=[
        gr.Dataframe(label="Generated Flashcards"),
        gr.File(label="Download CSV")
    ],
    title="TEN by10 – AI Flashcard Generator",
    description="Generate Anki flashcards with translations and example sentences."
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
